## Twitter Analysis

This Jupyter notebook uses the custom Python package we created to analyse our selected articles for engagement on Twitter. To do so, we have to start off by importing the Twitter utilities from that package, as well as some more stuff we will need throughout the process.

In [ ]:
from NewsTracker.Twitter import *
from NewsTracker.Utils import DelayedPrinter
import os

Next, we have to load our API keys for Twitter. These are stored, along all other API keys in a local file (".env"), which is not part of this repository, as the keys are private. Please refer to the "NewsTracker/Configuration.py" file to see how you can create your own config file.

Additionally, we will obtain an instance of the "TwitterAnalyzer", the core class that allows us optain and alalyse tweets for a given URL.

Finally, we create a "DelayedPrinter", a helpful little class that allows us to manage the way we print messages to the console more elegantly. It is only a conveniance tool though.

In [3]:
# load config
config = Configuration.load_from(".env")

# create twitter analyzer object
twitter = TwitterAnalyzer(config)

# create custom printer object for more pretty printing in console
printer = DelayedPrinter()

With that out of the way, we need to find a place to store our analysis results. For this purpose, we have two folders: The "final-results" folder will sort the results by topics and articles, while the "merged-results" are only sorted by topic. Additionally, we create two convenient functions, that allow us to create valid folder paths within these two folders, which we will use later to store files.

In [4]:
# defined folder names for results
result_dir = "./final-results/"
merged_dir = "./merged-results/"

# generate a foldername in result folder
def result_folder(foldername: str) -> str: 
    dir = os.path.join(result_dir, foldername)
    return dir if dir[-1] == "/" else dir + "/"

# generate a foldername in merged result folder
def merged_folder(foldername: str) -> str: 
    dir = os.path.join(merged_dir, foldername)
    return dir if dir[-1] == "/" else dir + "/"

Next, we need to provide a list of all topics, and the articles linked to them. We did so using a dictionary, which we can iterate through. This way, we also get the names of the topics, as well as the news outlets for sorting the results.

In [5]:
# dictionary for all topics and their respective articles
topics = {
    "elon-musk-twitter": {
        "bbc":                 "https://www.bbc.com/news/business-64010202",
        "euronews":            "https://www.euronews.com/next/2022/12/16/twitter-suspends-the-accounts-of-several-journalists-who-wrote-about-elon-musk",
        "cnbc":                "https://www.cnbc.com/2022/10/27/elon-musk-now-in-charge-of-twitter-ceo-and-cfo-have-left-sources-say.html",
        "wall-street-journal": "https://www.wsj.com/livecoverage/twitter-elon-musk-latest-news",
        "financial-times":     "https://www.ft.com/content/bb047c8f-f97d-4e3a-8bbb-50d0494c8c48",
        "guardian":            "https://www.theguardian.com/technology/2022/dec/17/elon-musk-reinstates-twitter-accounts-of-suspended-journalists"
    }, "worldcup": {
        "bbc":        "https://www.bbc.com/sport/football/63932622",
        "euronews-1": "https://www.euronews.com/2022/11/23/the-qatar-world-cup-kicks-off-with-an-extravagant-ceremony",
        "euronews-2": "https://www.euronews.com/2022/12/18/champions-lionel-messi-leads-argentina-to-electrifying-world-cup-victory-over-france",
        "cnbc":       "https://www.cnbc.com/2022/12/15/fubotv-hit-with-cyber-attack-during-world-cup-semifinal-match.html",
        "sky-spots":  "https://www.skysports.com/football/croatia-vs-morocco/report/463027",
        "fox-sports": "https://www.foxsports.com/stories/soccer/world-cup-2022-5-most-memorable-upsets-of-the-tourney",
        "guardian":   "https://www.theguardian.com/football/live/2022/dec/09/netherlands-v-argentina-world-cup-2022-quarter-final-live-score-updates"
    }, "chatgpt": {
        "guardian":          "https://www.theguardian.com/commentisfree/2022/dec/11/chatgpt-is-a-marvel-but-its-ability-to-lie-convincingly-is-its-greatest-danger-to-humankind",
        "bbc":               "https://www.bbc.com/news/technology-63861322",
        "euronews":          "https://www.euronews.com/next/2022/12/14/chatgpt-why-the-human-like-ai-chatbot-suddenly-got-everyone-talking",
        "cnbc":              "https://www.cnbc.com/2022/12/13/chatgpt-is-a-new-ai-chatbot-that-can-answer-questions-and-write-essays.html",
        "science-focus":     "https://www.sciencefocus.com/future-technology/gpt-3/",
        "bleeping-computer": "https://www.bleepingcomputer.com/news/technology/openais-new-chatgpt-bot-10-dangerous-things-its-capable-of/",
    }, "bts-army": {
        "bbc":                  "https://www.bbc.com/news/world-asia-63944860",
        "euronews":             "https://www.euronews.com/culture/2022/10/10/bts-south-korean-military-leader-says-he-wants-k-pop-band-to-do-military-service",
        "cnbc":                 "https://www.cnbc.com/2022/10/17/bts-stars-to-serve-military-duty-in-south-korea.html",
        "all-kpop":             "https://www.allkpop.com/article/2022/12/bts-jin-to-receive-extra-security-for-military-enlistment",
        "korea-joongang-daily": "https://koreajoongangdaily.joins.com/2022/12/13/entertainment/kpop/Korea-Jin-BTS/20221213094055711.html",
        "guardian":             "https://www.theguardian.com/music/2022/dec/15/bts-military-service-kpop-jin-kim-south-korea",
    }, "iran-women-rights": {
        "bbc":             "https://www.bbc.com/news/world-middle-east-62930425",
        "euronews":        "https://www.euronews.com/2022/12/04/iran-disbands-morality-police-amid-two-and-half-months-of-nationwide-protests",
        "cnbc":            "https://www.cnbc.com/2022/12/05/iran-denies-abolition-of-morality-police-as-three-day-strike-begins.html",
        "middle-east-eye": "https://www.middleeasteye.net/news/iran-mahsa-amini-protests-first-execution-carried-out",
        "arab-news":       "https://www.arabnews.com/node/2209306",
        "guardian":        "https://amp.theguardian.com/global-development/2022/dec/08/iranian-forces-shooting-at-faces-and-genitals-of-female-protesters-medics-say"
    }
}

Now, we construct a function that iterates over all topics and articles, creating a folder for each topic and article respectively. In these folders, it will then safe the results of the analysis produced by the "TwitterAnalyzer". This includes a serialization of the tweets itself.

In [6]:
# function to analyze the above articles
def analyze_topics():

    # iterate through all topics
    for topic, articles in topics.items():

        printer.delay(100 * "-")
        printer.delay(f"\nStart processing topic {topic}...")

        # iterate through all articles
        for index, (source, url) in enumerate(articles.items()):

            printer.delay(f"\nProcessing article {index + 1}/{len(articles)} now...\n")

            # analyze article and write result to respective folder
            dir = result_folder(f"{topic}/{source}")
            if not os.path.exists(dir):
                printer.print()
                twitter.analyze_url(url, dir)
            else:
                printer.pop()

        if printer.is_empty:
            print(f"\nFinished processing topic {topic}.\n")
            print(100 * "-")
            # exit()
        else:
            printer.clear()

However, we don't only want to look into the tweets sorted by articles, but also the results sorted by topics only, merging the tweets for seperate articles of the same topic. To do that, we can merge the results written to folders in the function above. The following function does that, creating a list of merged tweets, and analyzing them, writing the results in a new folder.

In [7]:
def merge_topics():
    
    # iterate through all topics
    for topic, articles in topics.items():

        parents = []
        tweets = []

        # iterate through att articles, and find the corresponding folders to read all tweets from them
        # merge all tweets from all articles in one list
        for source, url in articles.items():
            tweets_file = os.path.join(result_folder(f"{topic}/{source}"), "tweets.json")
            if os.path.exists(tweets_file):
                parent_tweets, all_tweets = twitter.load_tweets_from_file(tweets_file)
                parents += parent_tweets
                tweets += all_tweets

        # create merged results folder
        merged_folder_dir = merged_folder(f"{topic}")
        os.makedirs(merged_folder_dir, exist_ok=True)
        
        # analyze the merged tweets
        twitter.store_tweets(parents, os.path.join(merged_folder_dir, "tweets.json"))
        twitter.analyze(parents, tweets, merged_folder_dir)

Finally, we execute the both functions abaove.

In [8]:
analyze_topics()
merge_topics()

----------------------------------------------------------------------------------------------------

Start processing topic elon-musk-twitter...

Processing article 1/6 now...

Searching tweets using search term 'url:https://www.bbc.com/news/business-64010202'... 

c:\Users\maxim\Documents\Studium\Master\Year 2\Period 2\the-social-hack\NewsTracker\Twitter.py:84: RuntimeWarning: coroutine 'Twint.main' was never awaited
  break


Done.

Could not find any tweets.


Processing article 2/6 now...

Searching tweets using search term 'url:https://www.euronews.com/next/2022/12/16/twitter-suspends-the-accounts-of-several-journalists-who-wrote-about-elon-musk'... 

c:\Users\maxim\Documents\Studium\Master\Year 2\Period 2\the-social-hack\NewsTracker\Twitter.py:84: RuntimeWarning: coroutine 'Twint.main' was never awaited
  break


Done.

Could not find any tweets.


Processing article 3/6 now...

Searching tweets using search term 'url:https://www.cnbc.com/2022/10/27/elon-musk-now-in-charge-of-twitter-ceo-and-cfo-have-left-sources-say.html'... 

...

c:\Users\maxim\Documents\Studium\Master\Year 2\Period 2\the-social-hack\NewsTracker\Twitter.py:84: RuntimeWarning: coroutine 'Twint.main' was never awaited
  break


Done.

Could not find any tweets.


Processing article 6/6 now...

Searching tweets using search term 'url:https://amp.theguardian.com/global-development/2022/dec/08/iranian-forces-shooting-at-faces-and-genitals-of-female-protesters-medics-say'... 

c:\Users\maxim\Documents\Studium\Master\Year 2\Period 2\the-social-hack\NewsTracker\Twitter.py:84: RuntimeWarning: coroutine 'Twint.main' was never awaited
  break


Done.

Could not find any tweets.


Finished processing topic iran-women-rights.

----------------------------------------------------------------------------------------------------

Start analysis...

Extracting users...


NotFound: 404 Not Found
17 - No user matches for specified terms.